In [1]:
!pip install requests

In [1]:
import requests

In [6]:

# Set the base URL for your API
BASE_URL = 'http://127.0.0.1:8001/uploads/file_uploads'

# Define the GUID and file information
guid = 'test2.zip'  # Replace with an actual GUID from your database

# Headers for the requests
headers = {
    'Content-Type': 'application/json',
}


In [7]:

# Function to update the priority
def update_priority(guid, priority):    
    url = f'{BASE_URL}/update_priority/'
    data = {
        'guid': guid,
        'priority': priority,
    }
    response = requests.post(url, json=data, headers=headers)
    print('Update Priority Response:', response.json())


In [8]:

# Function to update the status
def update_status(guid, status):
    url = f'{BASE_URL}/update_status/'
    data = {
        'guid': guid,
        'status': status,
    }
    response = requests.post(url, json=data, headers=headers)
    print('Update Status Response:', response.json())


In [ ]:

# Test updating the priority
print("Testing Update Priority API")
update_priority(guid, 1)  # Set priority to 1


In [10]:

# Test pausing the upload
print("\nTesting Pause Upload API")
update_status(guid, 'paused')



Testing Pause Upload API
Update Status Response: {'status': 'status updated'}


In [ ]:

# Test resuming the upload
print("\nTesting Resume Upload API")
update_status(guid, 'resume')



In [ ]:

# # Test canceling the upload
# print("\nTesting Cancel Upload API")
# update_status(guid, 'cancel')

In [9]:
import pickle

serialized_data = b"\x80\x04\x95\xdb\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\tfile_path\x94\x8c\x12/tmp/test/test.zip\x94\x8c\x0bobject_name\x94\x8c\btest.zip\x94\x8c\x04guid\x94h\x04\x8c\x0cinstance_uid\x94\x8c\x11some_instance_uid\x94\x8c\bpriority\x94J\xe9\xb1xf\x8c\x06status\x94\x8c\x06queued\x94\x8c\ncreated_at\x94GA\xd9\x9e,z}[ \x8c\nupdated_at\x94GA\xd9\x9e,z}[6\x8c\ttimestamp\x94GA\xd9\x9e,z}[(\x8c\bprogress\x94K\x00u."

# Deserialize the data
data = pickle.loads(serialized_data)

# Print the resulting object
print(data)


{'file_path': '/tmp/test/test.zip', 'object_name': 'test.zip', 'guid': 'test.zip', 'instance_uid': 'some_instance_uid', 'priority': 1719185897, 'status': 'queued', 'created_at': 1719185897.9586868, 'updated_at': 1719185897.958692, 'timestamp': 1719185897.9586887, 'progress': 0}
